In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn.metrics import classification_report
from sklearn.preprocessing import OrdinalEncoder
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

import numpy as np
import plotly.express as px
import folium
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from folium import plugins
from sklearn.preprocessing import StandardScaler
from folium.plugins import HeatMap
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import os
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)


2024-05-10 01:42:51.545797: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
total_data = pd.read_csv('Taiwan_with_both.csv')
total_data = total_data[total_data["Total price (10,000 yuan)"]<3000]
total_data=total_data.reset_index().drop(columns="index")

#total_data.drop(columns=["Unnamed: 0","Unnamed: 0.1","Unnamed: 0.1.1","Unnamed: 0.2","Unnamed: 0.3","Unnamed: 0.4","Unnamed: 0.5"],axis=1,inplace=True)
print(len(total_data))
# split the DataFrame into 3 different subsets
trainmask = (total_data['transaction date'] < "110/01/01")
#validatemask = (total_data['transaction date'] >= "110/01/01") & (total_data['transaction date'] <= "110/03/31")
testmask = (total_data['transaction date'] >= "110/01/01")
train = total_data.loc[trainmask].copy()
test = total_data.loc[testmask].copy()
#validate = total_data.loc[validatemask].copy()

368349


In [4]:
train[train["parks_nearby_0_place_id"].isna()]

,Unnamed: 0,Lot location or house number,transaction date,"Total price (10,000 yuan)","Unit price (10,000 yuan/square meter)",Total area(ping),Proportion of main building,House age,management organization,elevator,City,geometry_lat,geometry_lng,Land,Construction,Car,Bedrooms,Living rooms,Bathrooms,Encoded_type,Encoded_floor,Encoded_purpose,Encoded_city,len_parks,parks_nearby_0_geometry_lat,parks_nearby_0_geometry_lng,parks_nearby_0_rating,parks_nearby_0_place_id,parks_nearby_0_user_ratings_total,parks_nearby_1_geometry_lat,parks_nearby_1_geometry_lng,parks_nearby_1_rating,parks_nearby_1_place_id,parks_nearby_1_user_ratings_total,parks_nearby_2_geometry_lat,parks_nearby_2_geometry_lng,parks_nearby_2_rating,parks_nearby_2_place_id,parks_nearby_2_user_ratings_total,parks_nearby_3_geometry_lat,parks_nearby_3_geometry_lng,parks_nearby_3_rating,parks_nearby_3_place_id,parks_nearby_3_user_ratings_total,parks_nearby_4_geometry_lat,parks_nearby_4_geometry_lng,parks_nearby_4_rating,parks_nearby_4_place_id,parks_nearby_4_user_ratings_total,len_hospitals,hospitals_nearby_0_geometry_lat,hospitals_nearby_0_geometry_lng,hospitals_nearby_0_rating,hospitals_nearby_0_place_id,hospitals_nearby_0_user_ratings_total,hospitals_nearby_1_geometry_lat,hospitals_nearby_1_geometry_lng,hospitals_nearby_1_rating,hospitals_nearby_1_place_id,hospitals_nearby_1_user_ratings_total,hospitals_nearby_2_geometry_lat,hospitals_nearby_2_geometry_lng,hospitals_nearby_2_rating,hospitals_nearby_2_place_id,hospitals_nearby_2_user_ratings_total,hospitals_nearby_3_geometry_lat,hospitals_nearby_3_geometry_lng,hospitals_nearby_3_rating,hospitals_nearby_3_place_id,hospitals_nearby_3_user_ratings_total,hospitals_nearby_4_geometry_lat,hospitals_nearby_4_geometry_lng,hospitals_nearby_4_rating,hospitals_nearby_4_place_id,hospitals_nearby_4_user_ratings_total
8,8,南區信義南街６９號六樓之一,104/07/01,550.0,15.5,35.58,60.59%,19.0,1,NaN,Taichung,24.127637,120.673423,3,1,0,3,2,2,0,1839,2,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,67,新莊區中平路１１０巷２８號十樓,104/07/01,1320.0,28.8,45.83,63.13%,24.0,1,NaN,New Taipei,25.045542,121.446306,1,1,0,3,2,1,0,2251,2,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
204,213,淡水區中正東路一段１３７號八樓之８,104/07/01,400.0,26.9,14.85,71.55%,10.0,1,NaN,New Taipei,25.159208,121.456276,1,1,0,1,1,1,4,1722,2,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
223,232,豐原區大明路２３６巷５弄８號,104/07/02,1560.0,0.0,54.34,NaN,25.0,0,NaN,Taichung,24.243859,120.707800,2,1,0,4,2,4,11,1669,2,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
346,357,太平區成功東路２９８號,104/07/02,500.0,13.0,38.34,85.51%,21.0,1,NaN,Taichung,24.130285,120.723845,1,1,0,3,2,2,0,430,0,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335127,350579,楊梅區金龍一路６號十二樓,109/12/27,518.0,16.0,32.45,56.96%,25.0,1,1.0,Taoyuan,24.444840,120.847897,1,1,0,2,2,1,0,2056,2,3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [5]:
# Encoding the string dataframe (Regionname), (Council) and (Suburb) columns
ordinal_enc = OrdinalEncoder()

#total_data['Encoded_type'] = ordinal_enc.fit_transform(total_data[['type']]).astype(int)
Encoded_type = total_data[["Encoded_type"]].drop_duplicates().sort_values(by='Encoded_type')

# total_data['Encoded_floor'] = ordinal_enc.fit_transform(total_data[['Floor type/floor height']]).astype(int)
Encoded_floor = total_data[["Encoded_floor"]].drop_duplicates().sort_values(by='Encoded_floor')

# total_data['Encoded_purpose'] = ordinal_enc.fit_transform(total_data[['The main purpose']]).astype(int)
Encoded_purpose = total_data[["Encoded_purpose"]].drop_duplicates().sort_values(by='Encoded_purpose').reset_index().drop(columns="index")
total_data = total_data.loc[total_data["Encoded_purpose"]>=0]
total_data = total_data.loc[total_data["Encoded_type"]>=0]
total_data = total_data.loc[total_data["Encoded_floor"]>=0]
total_data = total_data.reset_index().drop(columns="index")

#Encoded_floor = Encoded_floor.dropna(subset=["Floor type/floor height"]).reset_index().drop(columns="index")
#Encoded_purpose = Encoded_purpose.dropna(subset=["The main purpose"]).reset_index().drop(columns="index")
# Encoded_type.to_csv('../taiwan_dataset/v0/nodes_type_data.csv')
# Encoded_floor.to_csv('../taiwan_dataset/v0/nodes_floor_data.csv')
# Encoded_purpose.to_csv('../taiwan_dataset/v0/nodes_purpose_data.csv')


type_data = total_data["Encoded_type"].copy()
floor_data = total_data["Encoded_floor"].copy()
purpose_data = total_data["Encoded_purpose"].copy()

type_data = type_data.reset_index().rename(columns={"index":"node_id"})
floor_data = floor_data.reset_index().rename(columns={"index":"node_id"})
purpose_data = purpose_data.reset_index().rename(columns={"index":"node_id"})

# print(len(purpose_data))
# type_data = type_data[type_data["Encoded_type"]>=0]
# floor_data = floor_data[floor_data["Encoded_floor"]>=0]
# purpose_data = purpose_data[purpose_data["Encoded_purpose"]>=0]

print(len(purpose_data))

# type_data.to_csv('../taiwan_dataset/v0/edges_type_data.csv')
# floor_data.to_csv('../taiwan_dataset/v0/edges_floor_data.csv')
# purpose_data.to_csv('../taiwan_dataset/v0/edges_purpose_data.csv')

total_data['Encoded_city'] = ordinal_enc.fit_transform(total_data[['City']]).astype(int)
Encoded_city = total_data[["City", "Encoded_city"]].drop_duplicates().sort_values(by='Encoded_city').reset_index().drop(columns="index")
# Encoded_city["Encoded_city"].to_csv('../taiwan_dataset/v0/nodes_city_data.csv')
city_data = total_data["Encoded_city"].copy()
city_data = city_data.reset_index().rename(columns={"index":"node_id"})
city_data = city_data[city_data["Encoded_city"]>=0]

# city_data.to_csv('../taiwan_dataset/v0/edges_city_data.csv')

# total_data_removed =total_data.drop(columns=["type","Floor type/floor height","The main purpose", "Number of transactions", "The current layout of the building", "Remark"])
print(len(total_data))


368345
368345


In [6]:

df_concated=pd.concat([total_data[["parks_nearby_0_place_id"]].rename(columns={"parks_nearby_0_place_id":"a"}),total_data[["parks_nearby_1_place_id"]].rename(columns={"parks_nearby_1_place_id":"a"}),total_data[["parks_nearby_2_place_id"]].rename(columns={"parks_nearby_2_place_id":"a"}),total_data[["parks_nearby_3_place_id"]].rename(columns={"parks_nearby_3_place_id":"a"}),total_data[["parks_nearby_4_place_id"]].rename(columns={"parks_nearby_4_place_id":"a"})],ignore_index=True)
ordinal_enc = OrdinalEncoder()
res = ordinal_enc.fit_transform(df_concated)
res = pd.DataFrame(res.astype(int))
len_res = int(len(res)/5)
res

,0
0,5733
1,1451
2,4468
3,4402
4,1649
...,...
1841720,495
1841721,4710
1841722,4255
1841723,1077


In [7]:
total_data["parks_0_unique"] = res[:len_res]
total_data["parks_1_unique"] = res[len_res:len_res*2].reset_index().drop("index",axis=1)
total_data["parks_2_unique"] = res[len_res*2:len_res*3].reset_index().drop("index",axis=1)
total_data["parks_3_unique"] = res[len_res*3:len_res*4].reset_index().drop("index",axis=1)
total_data["parks_4_unique"] = res[len_res*4:len_res*5].reset_index().drop("index",axis=1)
Encoded_park_0 = total_data[["parks_nearby_0_geometry_lat","parks_nearby_0_geometry_lng","parks_nearby_0_rating", "parks_nearby_0_user_ratings_total", "parks_0_unique"]].drop_duplicates().sort_values(by='parks_0_unique')
Encoded_park_1 = total_data[["parks_nearby_1_geometry_lat","parks_nearby_1_geometry_lng","parks_nearby_1_rating", "parks_nearby_1_user_ratings_total", "parks_1_unique"]].drop_duplicates().sort_values(by='parks_1_unique')
Encoded_park_2 = total_data[["parks_nearby_2_geometry_lat","parks_nearby_2_geometry_lng","parks_nearby_2_rating", "parks_nearby_2_user_ratings_total", "parks_2_unique"]].drop_duplicates().sort_values(by='parks_2_unique')
Encoded_park_3 = total_data[["parks_nearby_3_geometry_lat","parks_nearby_3_geometry_lng","parks_nearby_3_rating", "parks_nearby_3_user_ratings_total", "parks_3_unique"]].drop_duplicates().sort_values(by='parks_3_unique')
Encoded_park_4 = total_data[["parks_nearby_4_geometry_lat","parks_nearby_4_geometry_lng","parks_nearby_4_rating", "parks_nearby_4_user_ratings_total", "parks_4_unique"]].drop_duplicates().sort_values(by='parks_4_unique')

In [8]:
Encoded_park_3

,parks_nearby_3_geometry_lat,parks_nearby_3_geometry_lng,parks_nearby_3_rating,parks_nearby_3_user_ratings_total,parks_3_unique
6,NaN,NaN,NaN,NaN,-9223372036854775808
36449,24.159993,120.711667,4.4,9.0,1
2092,24.172350,120.649038,4.1,2419.0,3
162598,24.172350,120.649038,4.1,2420.0,3
6157,24.990797,121.575093,4.8,8.0,5
...,...,...,...,...,...
185547,22.623298,120.353688,4.2,424.0,5727
55,25.030226,121.470976,3.9,735.0,5728
319,24.987968,121.451200,4.3,559.0,5733
1038,25.067760,121.665442,NaN,NaN,5735


In [9]:
Encoded_park_0.drop(index=8,inplace=True)
Encoded_park_1.drop(index=8,inplace=True)
Encoded_park_2.drop(index=6,inplace=True)
Encoded_park_3.drop(index=6,inplace=True)
Encoded_park_4.drop(index=6,inplace=True)

In [10]:
# Filling the missing rating values with 0
Encoded_park_0["parks_nearby_0_rating"].fillna((0), inplace=True)
Encoded_park_0["parks_nearby_0_user_ratings_total"].fillna((0), inplace=True)
Encoded_park_1["parks_nearby_1_rating"].fillna((0), inplace=True)
Encoded_park_1["parks_nearby_1_user_ratings_total"].fillna((0), inplace=True)
Encoded_park_2["parks_nearby_2_rating"].fillna((0), inplace=True)
Encoded_park_2["parks_nearby_2_user_ratings_total"].fillna((0), inplace=True)
Encoded_park_3["parks_nearby_3_rating"].fillna((0), inplace=True)
Encoded_park_3["parks_nearby_3_user_ratings_total"].fillna((0), inplace=True)
Encoded_park_4["parks_nearby_4_rating"].fillna((0), inplace=True)
Encoded_park_4["parks_nearby_4_user_ratings_total"].fillna((0), inplace=True)
save0_train = Encoded_park_0.rename(columns={"parks_nearby_0_place_id":"a","parks_nearby_0_geometry_lng":"b","parks_nearby_0_rating":"c", "parks_nearby_0_user_ratings_total":"d", "parks_0_unique":"e","parks_nearby_0_geometry_lat":"f"}).copy()
save1_train = Encoded_park_1.rename(columns={"parks_nearby_1_place_id":"a","parks_nearby_1_geometry_lng":"b","parks_nearby_1_rating":"c", "parks_nearby_1_user_ratings_total":"d", "parks_1_unique":"e","parks_nearby_1_geometry_lat":"f"}).copy()
save2_train = Encoded_park_2.rename(columns={"parks_nearby_2_place_id":"a","parks_nearby_2_geometry_lng":"b","parks_nearby_2_rating":"c", "parks_nearby_2_user_ratings_total":"d", "parks_2_unique":"e","parks_nearby_2_geometry_lat":"f"}).copy()
save3_train = Encoded_park_3.rename(columns={"parks_nearby_3_place_id":"a","parks_nearby_3_geometry_lng":"b","parks_nearby_3_rating":"c", "parks_nearby_3_user_ratings_total":"d", "parks_3_unique":"e","parks_nearby_3_geometry_lat":"f"}).copy()
save4_train = Encoded_park_4.rename(columns={"parks_nearby_4_place_id":"a","parks_nearby_4_geometry_lng":"b","parks_nearby_4_rating":"c", "parks_nearby_4_user_ratings_total":"d", "parks_4_unique":"e","parks_nearby_4_geometry_lat":"f"}).copy()
saveall_df_train = pd.concat([save0_train,save1_train,save2_train,save3_train,save4_train], ignore_index=True).drop_duplicates(subset='e').sort_values(by='e')
saveall_df_train

,f,b,c,d,e
0,24.866063,120.996006,0.0,0.0,0
2229,24.159993,120.711667,4.4,9.0,1
5230,23.959310,120.573765,0.0,0.0,2
1,24.172350,120.649038,4.1,2419.0,3
3,24.054547,120.696884,4.4,17.0,4
...,...,...,...,...,...
2228,24.987968,121.451200,4.3,559.0,5733
16334,25.123837,121.501850,3.8,6.0,5734
5228,25.067760,121.665442,0.0,0.0,5735
12378,24.185478,120.680420,4.5,2.0,5736


In [11]:
scaler_lat = MinMaxScaler(feature_range = (1,10))
scaler_lat.fit(train["geometry_lat"].values.reshape(-1, 1))
scaler_lng = MinMaxScaler(feature_range = (1,10))
scaler_lng.fit(train["geometry_lng"].values.reshape(-1, 1))
scaler_0 = MinMaxScaler(feature_range = (1,10))
saveall_df_train['f'] = scaler_lat.transform(saveall_df_train['f'].values.reshape(-1, 1))
saveall_df_train['b'] = scaler_lng.transform(saveall_df_train['b'].values.reshape(-1, 1))
saveall_df_train['d'] = scaler_0.fit_transform(saveall_df_train['d'].values.reshape(-1, 1))
saveall_df_train['01'] = 0
saveall_df_train['02'] = 0
saveall_df_train['03'] = 0
saveall_df_train['04'] = 0
saveall_df_train['05'] = 0
saveall_df_train['06'] = 0
saveall_df_train['07'] = 0
saveall_df_train['08'] = 0
saveall_df_train['09'] = 0
# saveall_df_train['10'] = 0
# saveall_df_train['11'] = 0
# saveall_df_train['12'] = 0
# saveall_df_train['13'] = 0
# saveall_df_train['14'] = 0
# saveall_df_train['15'] = 0
# saveall_df_train['16'] = 0
# saveall_df_train['17'] = 0
# saveall_df_train['18'] = 0

saveall_df_train

,f,b,c,d,e,01,02,03,04,05,06,07,08,09
0,1.759057,8.742584,0.0,1.000000,0,0,0,0,0,0,0,0,0,0
2229,1.571881,8.724857,4.4,1.001751,1,0,0,0,0,0,0,0,0,0
5230,1.518680,8.716259,0.0,1.000000,2,0,0,0,0,0,0,0,0,0
1,1.575156,8.720952,4.1,1.470572,3,0,0,0,0,0,0,0,0,0
3,1.543927,8.723935,4.4,1.003307,4,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2228,1.791374,8.770962,4.3,1.108743,5733,0,0,0,0,0,0,0,0,0
16334,1.827392,8.774120,3.8,1.001167,5734,0,0,0,0,0,0,0,0,0
5228,1.812526,8.784319,0.0,1.000000,5735,0,0,0,0,0,0,0,0,0
12378,1.578637,8.722909,4.5,1.000389,5736,0,0,0,0,0,0,0,0,0


In [12]:
saveall_df_train['feat'] = saveall_df_train.drop("e",axis=1).astype(str).agg(', '.join, axis=1)
saveall_df_train = saveall_df_train[["e", "feat"]]
saveall_df_train

,e,feat
0,0,"1.7590573638055762, 8.742583525611973, 0.0, 1...."
2229,1,"1.5718805712538657, 8.724856663248612, 4.4, 1...."
5230,2,"1.518680107995766, 8.716259288587048, 0.0, 1.0..."
1,3,"1.5751562375611705, 8.720952102784269, 4.1, 1...."
3,4,"1.5439271141956423, 8.72393506797663, 4.4, 1.0..."
...,...,...
2228,5733,"1.7913738074418974, 8.77096218119127, 4.3, 1.1..."
16334,5734,"1.827392245759687, 8.774119921804774, 3.8, 1.0..."
5228,5735,"1.8125264759720094, 8.784318884959387, 0.0, 1...."
12378,5736,"1.5786365584542592, 8.722908641117769, 4.5, 1...."


In [13]:
saveall_df_train.to_csv("nodes_parks_data.csv")

In [14]:
park_data_0 = total_data["parks_0_unique"].copy()
park_data_1 = total_data["parks_1_unique"].copy()
park_data_2 = total_data["parks_2_unique"].copy()
park_data_3 = total_data["parks_3_unique"].copy()
park_data_4 = total_data["parks_4_unique"].copy()

park_data_0 = park_data_0.reset_index().rename(columns={"index":"node_id"})
park_data_1 = park_data_1.reset_index().rename(columns={"index":"node_id"})
park_data_2 = park_data_2.reset_index().rename(columns={"index":"node_id"})
park_data_3 = park_data_3.reset_index().rename(columns={"index":"node_id"})
park_data_4 = park_data_4.reset_index().rename(columns={"index":"node_id"})

park_data_0 = park_data_0[park_data_0["parks_0_unique"]>=0]
park_data_1 = park_data_1[park_data_1["parks_1_unique"]>=0]
park_data_2 = park_data_2[park_data_2["parks_2_unique"]>=0]
park_data_3 = park_data_3[park_data_3["parks_3_unique"]>=0]
park_data_4 = park_data_4[park_data_4["parks_4_unique"]>=0]

park_data_0 = park_data_0.rename(columns={"parks_0_unique":"dest_id"}).copy()
park_data_1 = park_data_1.rename(columns={"parks_1_unique":"dest_id"}).copy()
park_data_2 = park_data_2.rename(columns={"parks_2_unique":"dest_id"}).copy()
park_data_3 = park_data_3.rename(columns={"parks_3_unique":"dest_id"}).copy()
park_data_4 = park_data_4.rename(columns={"parks_4_unique":"dest_id"}).copy()

saveall_df = pd.concat([park_data_0,park_data_1,park_data_2,park_data_3,park_data_4], ignore_index=True)
saveall_df

,node_id,dest_id
0,0,5733
1,1,1451
2,2,4468
3,3,4402
4,4,1649
...,...,...
1787967,368340,495
1787968,368341,4710
1787969,368342,4255
1787970,368343,1077


In [15]:
park_data_0

,node_id,dest_id
0,0,5733
1,1,1451
2,2,4468
3,3,4402
4,4,1649
...,...,...
368340,368340,2310
368341,368341,734
368342,368342,5022
368343,368343,5191


In [16]:
saveall_df.to_csv('edges_park_data.csv')


In [17]:
saveall_df

,node_id,dest_id
0,0,5733
1,1,1451
2,2,4468
3,3,4402
4,4,1649
...,...,...
1787967,368340,495
1787968,368341,4710
1787969,368342,4255
1787970,368343,1077
